In [2]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

In [4]:
# Get MNIST fashion dataset
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [19]:
# Model builder 
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    hp_units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units2, activation='relu'))
    model.add(keras.layers.Dense(10))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

    return model

In [20]:
# Tune the model 
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt', overwrite=True)

# Early stop for validiation loss
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Begin tuning
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

Trial 30 Complete [00h 00m 25s]
val_accuracy: 0.8829166889190674

Best val_accuracy So Far: 0.8853333592414856
Total elapsed time: 00h 05m 56s
INFO:tensorflow:Oracle triggered exit


In [22]:
# Get tuned values 
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and for the second densely-connected
layer is {best_hps.get('units2')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 320 and for the second densely-connected
layer is 96 and the optimal learning rate for the optimizer
is 0.001.



In [23]:
# Train model 
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4909 - accuracy: 0.8239 - val_loss: 0.4152 - val_accuracy: 0.8488
Epoch 2/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3684 - accuracy: 0.8641 - val_loss: 0.3763 - val_accuracy: 0.8648
Epoch 3/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3291 - accuracy: 0.8786 - val_loss: 0.3560 - val_accuracy: 0.8677
Epoch 4/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3052 - accuracy: 0.8862 - val_loss: 0.3497 - val_accuracy: 0.8784
Epoch 5/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2849 - accuracy: 0.8931 - val_loss: 0.3208 - val_accuracy: 0.8860
Epoch 6/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.2730 - accuracy: 0.8986 - val_loss: 0.3371 - val_accuracy: 0.8778
Epoch 7/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2583 - accuracy: 0.9035 - val_loss: 0.3379 - val_accuracy:

In [24]:
# retrain with best epoch 
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/34
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4915 - accuracy: 0.8248 - val_loss: 0.3949 - val_accuracy: 0.8565
Epoch 2/34
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3677 - accuracy: 0.8651 - val_loss: 0.3565 - val_accuracy: 0.8689
Epoch 3/34
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3284 - accuracy: 0.8791 - val_loss: 0.3684 - val_accuracy: 0.8698
Epoch 4/34
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3053 - accuracy: 0.8859 - val_loss: 0.3253 - val_accuracy: 0.8834
Epoch 5/34
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2867 - accuracy: 0.8935 - val_loss: 0.3391 - val_accuracy: 0.8757
Epoch 6/34
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2726 - accuracy: 0.8980 - val_loss: 0.3398 - val_accuracy: 0.8783
Epoch 7/34
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2606 - accuracy: 0.9020 - val_loss: 0.3251 - val_accuracy:

In [25]:
# Evaluate
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

'''
If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional overwrite=True argument while instantiating the tuner.
'''

313/313 [==============================] - 0s 1ms/step - loss: 0.5863 - accuracy: 0.8831
[test loss, test accuracy]: [0.5862670540809631, 0.8830999732017517]


'\nIf you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional overwrite=True argument while instantiating the tuner.\n'